In [1]:
#Loading Packages

from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import date, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
# Ezek nekem nem kellettek, mert újabb a seleniumom.
#from webdriver_manager.firefox import GeckoDriverManager
#from selenium.webdriver.common.by import By
import time
import pickle
import tqdm
from urllib.request import urlopen

In [30]:
# A legújabb verziójú seleniumon nálam így futott le, Huba, de te hagyd meg az előzőt, ha azzal ment! Az importokat is tartsd meg eredeti formájában
# Creating a driver - a legújabbon nálam így futott le, Huba, de te hagyd meg az előzőt!

driver = webdriver.Firefox()

In [4]:
#link

#https://index.hu/24ora/?s=Putyin&tol=2019-08-24&ig=2023-08-24&profil=&rovat=&cimke=h%C3%A1bor%C3%BA&word=1&pepe=1#
# "Putyin" - 2019.08.24. -- 2023.08.24. es időtartamban  háború címkével ellátva



In [31]:
# The driver gets the link

driver.get("https://index.hu/24ora/?s=Putyin&tol=2019-08-24&ig=2023-08-24&profil=&rovat=&cimke=h%C3%A1bor%C3%BA&word=1&pepe=1#")

In [32]:
#finding cookies, and clicking into

element = driver.find_element(By.CLASS_NAME, "css-k8o10q")
element.click()

In [33]:
# finding the advertising element and clicking later

element = driver.find_element(By.ID, "onesignal-slidedown-cancel-button")
element.click()

In [34]:
# Height of scroll, while loading the page

last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    # scrolls down
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # Waits for the pagee to loead given in sec / better technique: `waitforpageload` if possible
    time.sleep(2)
    # calculating scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

## BeautifulSoup 

In [10]:
#loading the scraped webpage with BeautifulSoup

s = BeautifulSoup(driver.page_source, 'lxml')

In [50]:
#Making a list
    #Finding 'a' tags - which are links in html code
        #iterating through them, to load them to the link_list


links_list = list()
links = s.find_all('a')
for link in links:
    try:
        links_list.append(link['href'])
    except:
        continue

In [49]:
len(links_list)

0

In [70]:
# Appending with list comprehension
# Or this is maybe just filtering the links list

links_list = [i for i in links_list if i.startswith("https://index.hu/belfold") or
              i.startswith("https://index.hu/kulfold") or
              i.startswith("https://index.hu/eu") or
              i.startswith("https://index.hu/gazdasag") or
              i.startswith("https://index.hu/velemeny")]

In [71]:
links_list[:6]


['https://index.hu/kulfold/2023/08/24/haboru-orosz-ukran-konfliktus-oroszorszag-vlagyimir-putyin-jevgenyij-prigozsin/',
 'https://index.hu/kulfold/2023/08/24/haboru-orosz-ukran-konfliktus-oroszorszag-vlagyimir-putyin-jevgenyij-prigozsin/',
 'https://index.hu/kulfold/2023/08/24/haboru-orosz-ukran-konfliktus-oroszorszag-vlagyimir-putyin-jevgenyij-prigozsin/',
 'https://index.hu/kulfold/2023/08/24/haboru-orosz-ukran-konfliktus-oroszorszag-jevgenyij-prigozsin-repulogep-szerencsetlenseg/',
 'https://index.hu/kulfold/2023/08/24/haboru-orosz-ukran-konfliktus-oroszorszag-jevgenyij-prigozsin-repulogep-szerencsetlenseg/',
 'https://index.hu/kulfold/2023/08/24/haboru-orosz-ukran-konfliktus-oroszorszag-jevgenyij-prigozsin-repulogep-szerencsetlenseg/']

In [72]:
# Generating unique links list

links_list = list(set(links_list))

In [74]:
len(links_list)

1284

In [59]:
# Checking the links
len(links_list)


5071

In [75]:
# Pickling: It is a process where a Python object hierarchy is converted into a byte stream.

with open('index_links_list.pickle', 'wb') as index_links_list:
    pickle.dump(links_list, index_links_list)

    #This function is called to serialize an object hierarchy.

In [76]:
# Unpickling: It is the inverse of Pickling process where a byte stream is converted into an object hierarchy. 

with open('index_links_list.pickle', 'rb') as index_links_list:
 
    links_list = pickle.load(index_links_list)
    # This function is called to de-serialize a data stream.

In [77]:
#pandas lépést egyben megcsináljuk a linklistával

In [78]:
#egy oszlopos dataframe, mely oszlop a linkeket tartalmazza, és link a neve
df = pd.DataFrame({'link': links_list})

In [80]:
df.drop_duplicates()

,link
0,https://index.hu/kulfold/2021/12/29/kijev-oros...
1,https://index.hu/kulfold/2022/03/24/ljubov-nep...
2,https://index.hu/kulfold/2023/06/21/haboru-oro...
3,https://index.hu/kulfold/2022/12/23/haboru-oro...
4,https://index.hu/kulfold/2023/05/30/haboru-oro...
...,...
1279,https://index.hu/kulfold/2022/08/12/haboru-zap...
1280,https://index.hu/kulfold/2023/01/29/haboru-oro...
1281,https://index.hu/kulfold/2023/02/24/haboru-ukr...
1282,https://index.hu/kulfold/2022/05/06/putyin-els...


In [81]:
from tqdm.auto import tqdm  # for notebooks

In [82]:
tqdm.pandas()

In [83]:
def fetch_html(link):
    return requests.get(link).content

def parse_html(html):
    s = BeautifulSoup(html, 'lxml')
    extract = lambda selector: s.find('div', {'class': selector}).get_text().strip() if s.find('div', {'class': selector}) else None

    return {
        "title": extract("content-title"),
        "author": extract("kovetes nem-kovetheto") or extract("kovetes"),
        "date": extract("datum"),
        "text": extract("cikk-torzs"),
        "lead": extract("lead")
    }

def get_article_data(link):
    html = fetch_html(link)
    data = parse_html(html) # ez egy dictionary
    data['html'] = html # html nevű key létrehozása, és ahhoz hozzáadva a html.
    return pd.Series(data) # ha Series-t return-öl ebben a formában, abból DataFrame lesz a végén

In [84]:
df = pd.concat([df['link'].progress_apply(get_article_data), df],axis=1)

  0%|          | 0/1284 [00:00<?, ?it/s]

100%|██████████| 1284/1284 [11:55<00:00,  1.80it/s]


In [85]:
df.head()

,title,author,date,text,lead,html,link
0,Orosz támadásra készül a kijevi polgármester h...,Kovács Ferenc,2021.12.29. 19:08,A kijevi lakosoknak sürgősen gondoskodniuk kel...,None,"b'<!DOCTYPE html>\n<html class=""no-js"" lang=""h...",https://index.hu/kulfold/2021/12/29/kijev-oros...
1,Ukrán nagykövet az Indexnek: Putyin nem áll me...,Bendarzsevszkij Anton,2022.03.24. 13:23\nMódosítva: 2022.03.24. 13:23,Egy hónapja tart az Oroszország által február ...,Oroszország Ukrajna ellen indított háborújának...,"b'<!DOCTYPE html>\n<html class=""no-js"" lang=""h...",https://index.hu/kulfold/2022/03/24/ljubov-nep...
2,Vlagyimir Putyin hadrendbe állítja a rettegett...,Index,,None,None,"b'<!DOCTYPE html>\n<html class=""no-js"" lang=""h...",https://index.hu/kulfold/2023/06/21/haboru-oro...
3,"A háború csúcsfegyvere, amelyet világszerte ak...",Faragó István,2022.12.23. 20:04,"Az elmúlt évek, így a 2022-es esztendő egyik l...",A harcászati drónokat sokkal költséghatékonyab...,"b'<!DOCTYPE html>\n<html class=""no-js"" lang=""h...",https://index.hu/kulfold/2022/12/23/haboru-oro...
4,„Ti büdös disznók!” – kemény támadás érte az o...,Bereznay István,2023.05.30. 21:09,Május 30-án hajnalban több tucat drón (egyesek...,Kedd hajnalban dróntámadás érte az orosz fővár...,"b'<!DOCTYPE html>\n<html class=""no-js"" lang=""h...",https://index.hu/kulfold/2023/05/30/haboru-oro...


In [29]:
df.tail()

,title,author,date,text,lead,html,link,title_cleaned
132,Orbán Viktor az új kormány összetételéről és a...,Németh Márton Sándor,2022.04.06. 16:45,Orbán Viktor három pontban összegezte a Fidesz...,A vasárnapi választási győzelem után Orbán Vik...,"b'<!DOCTYPE html>\n<html class=""no-js"" lang=""h...",https://index.hu/belfold/2022/04/06/orban-vikt...,Orbán Viktor az új kormány összetételéről és a...
133,Szakály Sándor: Vlagyimir Putyin nem megy tová...,Bereznay István,2022.04.04. 17:53,A történész kiemelte: a konfliktus még a szovj...,"„Úgy gondolom, Vlagyimir Putyin nem megy továb...","b'<!DOCTYPE html>\n<html class=""no-js"" lang=""h...",https://index.hu/belfold/2022/04/04/haboru-oro...,Szakály Sándor: Vlagyimir Putyin nem megy tová...
134,Mit mondott a kormányoldal és az ellenzék a ka...,Sz.Z.,2022.04.02. 12:26\nMódosítva: 2022.04.02. 12:26,A kormány szerint a tét: háború vagy béke\nOrb...,Abban a politikai paletta valamennyi szereplőj...,"b'<!DOCTYPE html>\n<html class=""no-js"" lang=""h...",https://index.hu/belfold/2022/04/02/orban-mark...,Mit mondott a kormányoldal és az ellenzék a ka...
135,Kiállítás nyílt az ukrán menekültek fotóiból Ú...,Varga Maja Bernadett,2022.03.29. 17:03,"A fővárosi Móricz Zsigmond körtéren tartott, o...",Kiállítás nyílt az ukrán menekültek fotóiból Ú...,"b'<!DOCTYPE html>\n<html class=""no-js"" lang=""h...",https://index.hu/belfold/2022/03/29/haboru-oro...,Kiállítás nyílt az ukrán menekültek fotóiból Ú...
136,Márki-Zay Péter: Orbán Viktor a Nyugat árulója...,- nms -,2022.02.24. 09:54,"Ahogy arról beszámoltunk, Oroszország megtámad...",Az ellenzék miniszterelnök-jelöltje keményen k...,"b'<!DOCTYPE html>\n<html class=""no-js"" lang=""h...",https://index.hu/belfold/2022/02/24/marki-zay-...,Márki-Zay Péter: Orbán Viktor a Nyugat árulója...


In [86]:
df['title_cleaned'] = df['title'].apply(lambda text: text.replace("\n", "") if isinstance(text, str) else text)
df.to_csv('putin_full.csv')

In [69]:
import os
os.getcwd()

'c:\\Users\\userű\\Downloads'

In [25]:
brusszel_df_1 = pd.read_csv('C:/Users/userű/Downloads/cleaned_title_list.csv', header=0)


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/userű/Downloads/cleaned_title_list.csv'

In [ ]:
#COMMENTS

 # Python packages for text mining

    #nrclex a modell
    #vader - pretty good
    #glove model